In [1]:
import pandas as pd
import numpy as np

class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.freeagents = freeagents
        self.turns = turns
        self.playerJustMoved = playerjm

class MLBPlayer:
    def __init__(self, name, team, position, points):
        self.name = name
        self.team = team
        self.position = str(position)
        self.points = points
    def __repr__(self):
        return self.name
    def pop(self, index=-1) :
        return self.data.pop(index)

def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    if playerjm is None: return 0
    
    pos_wgts = {
        ("IF"): [1],
        ("OF"): [1],
        ("P"): [1]
        # ("IF", "OF"):[1]
        # ("SS"): [1],
        # ('3B'): [1],
        # # ("1B", "3B"): [.6, .4],
        # # ("2B", "SS"): [.6, .4],
        # # ('LF'):[.3],
        # # ('CF'):[.3],
        # ('OF'):[1],
        # ("C", "1B", "2B", "SS", "3B", 'DH'): [1],
        # ("SP"): [1],
        # ("RP"): [1]
    }
    result = 0
    # map the drafted players to the weights
    for p in self.rosters[playerjm]:
        max_wgt, _, max_pos, old_wgts = max(
            ((wgts[0], -len(lineup_pos), lineup_pos, wgts) for lineup_pos, wgts in pos_wgts.items()
                if p.position in lineup_pos),
            default=(0, 0, (), []))
        if max_wgt > 0:
            result += max_wgt * p.points
            old_wgts.pop(0)
            if not old_wgts:
                pos_wgts.pop(max_pos)
                
    # map the remaining weights to the top three free agents
    for pos, wgts in pos_wgts.items():
        result += np.mean([p.points for p in self.freeagents if p.position in pos][:3]) * sum(wgts)
    return result
DraftState.GetResult = GetResult

def GetMoves(self):
    """ Get all possible moves from this state.
    """
    
    pos_max = {"IF": 7, "OF": 7, "P": 7}
    if len(self.turns) == 0: return []
    roster_positions = np.array([p.position for p in self.rosters[self.turns[0]]], dtype=str)
    # moves = [pos for pos, max_ in pos_max.items() if np.sum(roster_positions == pos) < max_]
    moves = list(pos_max.keys())
    return moves
DraftState.GetMoves = GetMoves

def DoMove(self, move):
    """ Update a state by carrying out the given move.
        Must update playerJustMoved.
    """
    player = next(p for p in self.freeagents if p.position == move)
    self.freeagents.remove(player)
    rosterId = self.turns.pop(0)
    self.rosters[rosterId].append(player)
    self.playerJustMoved = rosterId
    
DraftState.DoMove = DoMove

def Clone(self):
    """ Create a deep clone of this game state.
    """
    rosters = list(map(lambda r: r[:], self.rosters))
    st = DraftState(rosters, self.turns[:], self.freeagents[:],
            self.playerJustMoved)
    return st
DraftState.Clone = Clone

# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai
from math import *
import random
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        UCTK = 200
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result
def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
    """
    rootnode = Node(state = rootstate)
    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()
        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)
        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree
        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))
        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode
    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

# Read in data

In [26]:
mlb_players = pd.read_csv("~/Documents/GitHub/fantasy_baseball_draft_bot/data/mcts_players.csv")

freeagents = [MLBPlayer(*p) for p in mlb_players.itertuples(index=False, name=None)]

num_competitors = 12
rosters = [[] for _ in range(num_competitors)] # empty rosters to start with

In [27]:
num_rounds = 20
turns = []
# generate turns by snake order
for i in range(num_rounds):
    turns += reversed(range(num_competitors)) if i % 2 else range(num_competitors)
    
state = DraftState(rosters, turns, freeagents)
iterations = 1000
while state.GetMoves() != []:
    move = UCT(state, iterations)
    state.DoMove(move)
    print((state.rosters[state.playerJustMoved][-1], move), end=".")


(Aaron Judge, 'OF').(Juan Soto, 'OF').(Ronald Acuña Jr., 'OF').(Gerrit Cole, 'P').(Kyle Tucker, 'OF').(Mike Trout, 'OF').(Mookie Betts, 'OF').(Julio Rodríguez, 'OF').(Shohei Ohtani, 'OF').(Yordan Alvarez, 'OF').(Kyle Schwarber, 'OF').(Freddie Freeman, 'IF').(Corbin Burnes, 'P').(José Ramírez, 'IF').(Vladimir Guerrero Jr., 'IF').(Pete Alonso, 'IF').(Trea Turner, 'IF').(Paul Goldschmidt, 'IF').(Matt Olson, 'IF').(Manny Machado, 'IF').(Bryan Reynolds, 'OF').(Aaron Nola, 'P').(Bo Bichette, 'IF').(Luis Castillo, 'P').(Austin Riley, 'IF').(Dylan Cease, 'P').(Rafael Devers, 'IF').(Marcus Semien, 'IF').(Shane Bieber, 'P').(Sandy Alcantara, 'P').(Robbie Ray, 'P').(Spencer Strider, 'P').(Kevin Gausman, 'P').(Brandon Woodruff, 'P').(Framber Valdez, 'P').(Randy Arozarena, 'OF').(Cedric Mullins, 'OF').(Bobby Witt Jr., 'IF').(George Springer, 'OF').(Christian Yelich, 'OF').(Max Scherzer, 'P').(Shane McClanahan, 'P').(Alex Bregman, 'IF').(Michael Harris II, 'OF').(Jose Altuve, 'IF').(Cristian Javier,

KeyboardInterrupt: 

In [31]:
tm1 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[0])])
tm2 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[1])])
tm3 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[2])])
tm4 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[3])])
tm5 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[4])])
tm6 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[5])])
tm7 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[6])])
tm8 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[7])])
tm9 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[8])])
tm10 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[9])])
tm11 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[10])])
tm12 = pd.DataFrame([r.name for i, r in enumerate(state.rosters[11])])
results = pd.concat([tm1, tm2, tm3, tm4, tm5, tm6, tm7, tm8, tm9, tm10, tm11, tm12], axis= 1)
results.columns = ['tm1', 'tm2', 'tm3', 'tm4', 'tm5', 'tm6', 'tm7', 'tm8', 'tm9', 'tm10', 'tm11', 'tm12']
results

,tm1,tm2,tm3,tm4,tm5,tm6,tm7,tm8,tm9,tm10,tm11,tm12
0,Aaron Judge,Juan Soto,Ronald Acuña Jr.,Gerrit Cole,Kyle Tucker,Mike Trout,Mookie Betts,Julio Rodríguez,Shohei Ohtani,Yordan Alvarez,Kyle Schwarber,Freddie Freeman
1,Luis Castillo,Bo Bichette,Aaron Nola,Bryan Reynolds,Manny Machado,Matt Olson,Paul Goldschmidt,Trea Turner,Pete Alonso,Vladimir Guerrero Jr.,José Ramírez,Corbin Burnes
2,Austin Riley,Dylan Cease,Rafael Devers,Marcus Semien,Shane Bieber,Sandy Alcantara,Robbie Ray,Spencer Strider,Kevin Gausman,Brandon Woodruff,Framber Valdez,Randy Arozarena
3,Francisco Lindor,Teoscar Hernández,Cristian Javier,Jose Altuve,Michael Harris II,Alex Bregman,Shane McClanahan,Max Scherzer,Christian Yelich,George Springer,Bobby Witt Jr.,Cedric Mullins
4,Brandon Nimmo,Corey Seager,Adolis García,Taylor Ward,Anthony Santander,Tyler O'Neill,Alek Manoah,Yu Darvish,Eloy Jiménez,Ian Happ,Zack Wheeler,Justin Verlander
5,Kris Bryant,Corbin Carroll,Jacob deGrom,Logan Webb,Julio Urías,Hunter Greene,Triston McKenzie,Nolan Arenado,Max Fried,Luis Robert Jr.,Zac Gallen,Steven Kwan
6,Rhys Hoskins,Lucas Giolito,Charlie Morton,Nick Castellanos,Xander Bogaerts,Dansby Swanson,Logan Gilbert,Nick Lodolo,Willy Adames,Brady Singer,Daulton Varsho,Kyle Wright
7,J.D. Martinez,Blake Snell,Fernando Tatis Jr.,Alex Verdugo,Byron Buxton,Will Smith,Joe Musgrove,Lance Lynn,Pablo López,Starling Marte,Chris Bassitt,Jake McCarthy
8,Seiya Suzuki,Hunter Renfroe,Carlos Correa,Matt Chapman,Joe Ryan,Riley Greene,Merrill Kelly,Jordan Montgomery,José Abreu,George Kirby,Christian Walker,Patrick Sandoval
9,Joey Meneses,Nick Pivetta,Carlos Rodón,Miles Mikolas,Wander Franco,Luis Severino,Reid Detmers,José Berríos,Max Muncy,Jon Gray,Germán Márquez,Nestor Cortes
